In [1]:
import Data.Monoid
import Control.Monad
import Test.QuickCheck
import Test.QuickCheck.Checkers
import Test.QuickCheck.Classes

<h3>New Maybe type</h3>

In [ ]:
data Optional a = Nada | Only a deriving (Eq, Show)

In [ ]:
instance Semigroup a => Semigroup (Optional a) where
    (<>) Nada a = a
    (<>) a Nada = a
    (<>) (Only a) (Only b) = Only (a <> b)

In [ ]:
instance Monoid a => Monoid (Optional a) where
    mempty = Only mempty
    mappend = (<>)

<h3>Quick Check examples</h3>

In [ ]:
monoidAssoc :: (Eq m, Monoid m)
    => m -> m -> m -> Bool
monoidAssoc a b c = (a <> (b <> c)) == ((a <> b) <> c)

In [ ]:
type S = String
quickCheck (monoidAssoc :: S -> S -> S -> Bool)

In [ ]:
monoidLeftIdentity :: (Eq m, Monoid m)
    => m -> Bool
monoidLeftIdentity a = (mempty <> a) == a

monoidRightIdentity :: (Eq m, Monoid m)
    => m -> Bool
monoidRightIdentity a = (a <> mempty) == a

In [ ]:
quickCheck (monoidLeftIdentity :: S -> Bool)

In [ ]:
quickCheck (monoidRightIdentity :: S -> Bool)

In [2]:
data Bull = Fools | Twoo deriving (Eq, Show)

instance Arbitrary Bull where
    arbitrary = frequency [(1, return Fools), (1, return Twoo)]

instance Semigroup Bull where
    (<>) _ _ = Fools

instance Monoid Bull where
    mempty = Fools
    mappend _ _ = Fools

In [ ]:
type BullMappend = Bull -> Bull -> Bull -> Bool

In [ ]:
quickCheck (monoidAssoc :: BullMappend)

In [ ]:
quickCheck (monoidLeftIdentity :: Bull -> Bool)

In [3]:
instance EqProp Bull where
    (=-=) = eq

In [17]:
verboseBatch (monoid Twoo)


monoid:
  left  identity: *** Failed! Falsifiable (after 1 test):
Twoo
  right identity: *** Failed! Falsifiable (after 2 tests):
Twoo
  associativity:  +++ OK, passed 500 tests.

<h3>Maybe monoid without underlying monoid type</h3>

In [ ]:
newtype First' a = First' { getFirst' :: Optional a } deriving (Eq, Show)

In [ ]:
instance Arbitrary a => Arbitrary (First' a) where
    arbitrary = frequency [(1, return (First' Nada)), (3, fmap (First' . Only) arbitrary)]

In [ ]:
instance Semigroup (First' a) where
    (<>) _ (First' (Only a)) = First' (Only a)
    (<>) a (First' Nada) = a
    
instance Monoid (First' a) where
    mempty = First' Nada
    mappend = (<>)

In [ ]:
firstMappend :: First' a -> First' a -> First' a
firstMappend = mappend

In [ ]:
type FirstMappend = First' String -> First' String -> First' String -> Bool
type FstId = First' String -> Bool

In [ ]:
quickCheck(monoidAssoc :: FirstMappend)

In [ ]:
quickCheck(monoidLeftIdentity :: FstId)
quickCheck(monoidRightIdentity :: FstId)

<h3>List type</h3>

In [5]:
data Listy a = Nil | Cons {head':: a, tail':: Listy a} deriving (Eq, Show)

In [6]:
instance Arbitrary a => Arbitrary (Listy a) where
    arbitrary = frequency [(1, pure Nil), (5, Cons <$> arbitrary <*> arbitrary)]

In [7]:
instance Semigroup (Listy a) where
    (<>) Nil ys = ys
    (<>) (Cons x xs) ys = Cons x (xs <> ys)

In [8]:
instance Monoid (Listy a) where
    mempty = Nil
    mappend = (<>)

In [ ]:
type ListyMappend = Listy String -> Listy String -> Listy String -> Bool
type ListyId = Listy String -> Bool

In [ ]:
quickCheck(monoidAssoc :: ListyMappend)

In [ ]:
quickCheck(monoidLeftIdentity :: ListyId)
quickCheck(monoidRightIdentity :: ListyId)

In [10]:
instance Eq a => EqProp (Listy a) where
    (=-=) = eq

In [14]:
type LS = Listy String
trigger :: LS
trigger = undefined

In [16]:
verboseBatch (monoid trigger)


monoid:
  left  identity: +++ OK, passed 500 tests.
  right identity: +++ OK, passed 500 tests.
  associativity:  +++ OK, passed 500 tests.

<h3>Tupley (like Tuple)</h3>

In [ ]:
data Tupley a b = Tupley {first:: a, second:: b} deriving (Eq, Show)

instance (Arbitrary a, Arbitrary b) => Arbitrary (Tupley a b) where
    arbitrary = do
        a <- arbitrary
        b <- arbitrary
        return (Tupley a b)

In [ ]:
instance (Semigroup a, Semigroup b) => Semigroup (Tupley a b) where
    (<>) (Tupley a b) (Tupley a' b') = Tupley (a <> a') (b <> b')

In [ ]:
instance (Monoid a, Monoid b) => Monoid (Tupley a b) where
    mempty = Tupley mempty mempty
    mappend = (<>)

In [ ]:
type TupleyAssoc = Tupley (Sum Int) String -> Tupley (Sum Int) String -> Tupley (Sum Int) String -> Bool
type TupleyId = Tupley (Sum Int) String -> Bool

In [ ]:
quickCheck(monoidAssoc :: TupleyAssoc)

In [ ]:
quickCheck(monoidLeftIdentity :: TupleyId)
quickCheck(monoidRightIdentity :: TupleyId)